Testando se o ambiente está funcionando

In [2]:
print('teste')

teste


Importando bibliotecas para explorar os diretórios do computador

In [3]:
from os import listdir
from os.path import isfile, join

Verificando todos os arquivos obtidos da fonta de dados

In [32]:
path = "E://dump"

dump_files = listdir(path)

In [5]:
import pandas as pd

Tendo certeza que é possível ler os dados do GHtorrent.

In [6]:
#pd.read_csv(join(path, 'project_languages.csv'))

Acredito que será possível partir dos pullrequest para observar quais repositórios possuem contribuições ou não. Além disso, após pesquisar (https://levelup.gitconnected.com/how-to-mine-github-data-in-2022-e9c70b3f61d3) parece que os dados de 2015 possuem uma estrutura diferente. Então irei tentar filtrar eles fora do meu dataset

In [7]:
#df_pullrequests = pd.read_csv(join(path, 'pull_requests.csv'))

In [8]:
#df_pullrequests

Um artigo que mostra a estrutura dos arquivos do GHtorrent https://gousios.org/pub/ghtorrent-dataset-toolsuite.pdf
misturando com esse aqui também https://github.com/ghtorrent/ghtorrent.org/blob/master/files/schema.pdf
Então irei adicionar o nome nas colunas, acabei retirando a coluna pullrequest_id pois ela não fazia sentido no contexto

In [9]:
#df_pullrequests.columns = ["id", "head_repo_id", "base_repo_id", "head_commit_id", "base_commit_id", "intra_branch_id","merged"]

Como os pull requests não possuem data de criação tentarei filtrar através dos commits que eles estão ligados.

In [10]:
df_commits = pd.read_csv(join(path, 'commits.csv'), names=["id", "sha", "author_id", "commiter_id", "project_id", "created_at"], chunksize=5000000)

In [11]:
# # Função para calcular o número total de chunks
# def get_total_chunks(filename, chunksize):
#     # Contar o número total de linhas no arquivo
#     total_lines = sum(1 for _ in open(filename)) - 1  # Subtrai 1 por causa do cabeçalho
#     # Calcular o número total de chunks
#     total_chunks = (total_lines // chunksize) + (1 if total_lines % chunksize != 0 else 0)
#     return total_chunks

# # Exemplo de uso
# filename = join(path, 'commits.csv')
# chunksize = 5000000
# total_chunks = get_total_chunks(filename, chunksize)
# print(f"Total de chunks: {total_chunks}")

In [12]:
df_commits_19 = pd.DataFrame({},columns=["id", "sha", "author_id", "commiter_id", "project_id", "created_at"])



In [13]:
def checkDate(df):
   return pd.to_datetime(df.iloc[0]['created_at']) > pd.to_datetime('2019-01-01') or pd.to_datetime(df.iloc[-1]['created_at']) > pd.to_datetime('2019-01-01')




In [14]:
# for chunk in df_commits:
#     chunk.map(lambda row: chechRowDate(row))

In [15]:
# i=0

# for chunk in df_commits:
#     if checkDate(chunk):
#         chunk.to_csv(f'E:/dump/commits_greater_than_2k19_{i}')

#     i += 1

In [16]:
df_commits_19

,id,sha,author_id,commiter_id,project_id,created_at


Devido a grande quantidade de commits, não foi possível os utilizar como filtro de projetos open-source. Nesse caso, iremos utilizar a relação project_members para encontrar quais projetos são relevantes, ou seja, que possuem contribuintes além do dono.

In [17]:
df_project_members = pd.read_csv(join(path, 'project_members.csv'), names=["repo_id", "user_id", "created_at", "ext_ref_id"])

In [18]:
df_project_members

,repo_id,user_id,created_at,ext_ref_id
0,1,1,2009-12-08 10:17:27,NaN
1,1,2,2009-12-08 10:17:27,NaN
2,1,4,2009-12-08 10:17:27,NaN
3,3,24,2011-02-19 01:58:42,NaN
4,3,5465,2011-02-19 01:58:42,NaN
...,...,...,...,...
15525047,207191305,221488,2021-03-06 22:46:40,NaN
15525048,207191367,54196484,2021-03-06 22:51:54,NaN
15525049,207191455,54204581,2021-03-06 22:50:39,NaN
15525050,207191498,3518100,2021-03-06 22:45:27,NaN


In [19]:
df_count_members = df_project_members.groupby("repo_id", as_index=False)["user_id"].count()

In [20]:
df_count_members

,repo_id,user_id
0,1,3
1,3,19
2,5,1
3,6,6
4,8,16
...,...,...
9549778,207191305,1
9549779,207191367,1
9549780,207191455,1
9549781,207191498,1


In [24]:
df_open_source = df_count_members[df_count_members["user_id"]>10]

In [25]:
df_open_source

,repo_id,user_id
1,3,19
4,8,16
14,19,14
26,34,38
27,37,42
...,...,...
9544854,206859445,12
9545989,206918002,12
9546379,206955864,17
9547045,206997704,19


A partir do estudo Recommending GitHub Projects for Developer Onboarding podemos extrair alguns atributos a serem extraídos do dataset para que seja estudado o quão receptivo um repositório é, são eles:

- Crescimento do Projeto: Quantidade de commits do projeto no momento de iniciação
- Data de criação do projeto: Projetos mais novos tendem a atrair mais desenvolvedores
- Momento de primeira contribuição: quando o projeto começa a receber contribuidores
- Momento da última contribuição: se a equipe do projeto se manter estável significa que existe menos oportunidades de onboarding
- Momento do primeiro commit: primeiro commit do projeto, a partir de quando passa a aceitar contribuições
- Momento do último commit: se o projeto receber muitos commits, mas nao tiver tanto onboarding significa q esse projeto necessita de mais contribuintes

In [101]:
#objetivo final

df_onboardings = pd.DataFrame({}, columns=["repo_id", "created_at", "total_commits","first_onboarding", "last_onboarding", "first_commit", "last_commit", "user_id", "onboarding_date"])

df_onboardings

,repo_id,created_at,total_commits,first_onboarding,last_onboarding,first_commit,last_commit,user_id,onboarding_date


Aqui irei obter todos os usuários que já contribuiram pra algum projeto open-source

In [27]:
df_project_members = df_project_members[ df_project_members['repo_id'].isin(df_open_source['repo_id'])]

Devivo a datação dos dados iremos filtrar somente os membros a partir de 2019

In [30]:
df_project_members = df_project_members[pd.to_datetime(df_project_members['created_at']) > pd.to_datetime("01-01-2019")]

Agora obtemos todos os commits referentes aos projetos filtrados anteriormente

In [ ]:
all_open_source_commits = []

for file in dump_files:
    if "greater" in file:
        df_commit = pd.read_csv(join(path,file))
        df_commit = df_commit[df_commit['project_id'].isin(df_project_members["repo_id"])]

        all_open_source_commits.append(df_commit)

df_commits_open_source = pd.concat(all_open_source_commits)

acho que vou ter q inverter a ordem das coisas: primeiro pegar os commits de projetos open source, depois filtrar as primeiras contribuições dos projetos e só então filtrar por 2019

In [76]:
df_commits_open_source = df_commits_open_source.drop('Unnamed: 0', axis=1)

df_commits_open_source.to_csv(join(path,"df_commits_open_source.csv"))

In [87]:
#transforma o campo de data em data

df_commits_open_source['created_at'] = pd.to_datetime(df_commits_open_source['created_at'])

In [88]:
df_commits_open_source

,id,sha,author_id,commiter_id,project_id,created_at
4980791,1949661766,1d773837d4bd4f0da0a224eb8dac8208ab2d84fd,10658540,10658540,75015574,2020-11-10 08:36:16
4981125,1949662107,52aed20c41c4027a49be6a1be136ddb3e3bb9d38,58131281,30351173,187678775,2020-11-18 08:14:18
4981143,1949662126,58be51c3a4139a0982b2e1f8b74d29d19ccb8fc1,63737388,39254391,187678775,2020-11-18 07:39:45
4981157,1949662140,e8e29f547a81dd59d07e6246ff4a97448fc4cfaf,58131281,58131281,187678775,2020-11-18 08:13:52
4981178,1949662161,0e62dc652d27a9b1ab00577cee3e8869f44850e4,58880882,58880882,187678775,2020-11-18 16:33:04
...,...,...,...,...,...,...
4999409,1944555635,b2f62024dd828820d92fee49707d4f293a5a10d3,63582454,30351173,189590167,2020-11-11 16:33:19
4999419,1944555645,0779e438143ed221db28f53d77057d9318950060,64203951,30351173,189590167,2020-11-11 16:26:54
4999938,1944556174,af0d2085d456f697eb462b1e0bf6ce5ddfe14e99,40002355,30351173,7975648,2020-11-23 11:25:40
4999941,1944556177,e3e8e1dca222c23778c28ea6a3fc8081b7d5612d,40002355,40002355,7975648,2020-11-16 15:41:16


Agora iremos obter o resultado o primeiro commit de cada usuário

In [ ]:
def get_first_user_commit(row):
    df_commits_open_source[df_commits_open_source['author_id'] == row['user_id']]

In [94]:
unique_users = df_commits_open_source['author_id'].unique()

first_commit_user = []
last_commit_user = []


for user in unique_users:
    df_user_commit = df_commits_open_source[df_commits_open_source['author_id'] == user]
    first_commit_user.append(df_user_commit.iloc[df_user_commit['created_at'].argmin()])
    last_commit_user.append(df_user_commit.iloc[df_user_commit['created_at'].argmax()])

Agora que temos os projetos que possuem contribuidores podemos utilizar para filtrar a relação projetos

In [98]:
df_first_commit_user = pd.DataFrame(first_commit_user)

df_first_commit_user

,id,sha,author_id,commiter_id,project_id,created_at
3018822,1732535513,cede9aa633d7ad7b608ea27a19de24b9709e0d9f,10658540,10658540,68148053,2019-05-12 03:21:28
4981157,1949662140,e8e29f547a81dd59d07e6246ff4a97448fc4cfaf,58131281,58131281,187678775,2020-11-18 08:13:52
4981143,1949662126,58be51c3a4139a0982b2e1f8b74d29d19ccb8fc1,63737388,39254391,187678775,2020-11-18 07:39:45
4355831,1933778312,efdd3f4e4cf001c4dc983c11f62fec524c6b6c37,58880882,58880882,187678775,2020-11-04 23:23:42
536676,1714119731,ed7b09ecdefdb870495b88275b0346267eed7e97,31387939,31387939,57395751,2020-07-01 14:04:04
...,...,...,...,...,...,...
4996312,1944552472,292ee700dd47991d565136c7f9c833084e908b67,64250705,30351173,87128207,2020-11-16 10:35:28
4998845,1944555054,4abcd70a283158c43aa420fe31a8283ad281a7d3,12323404,12323404,188986753,2020-11-05 21:48:59
4999380,1944555606,f3f14b65313e6f8ed7dc6790c9a2df3c2086a299,64062281,30351173,189590167,2020-11-11 16:24:25
4999409,1944555635,b2f62024dd828820d92fee49707d4f293a5a10d3,63582454,30351173,189590167,2020-11-11 16:33:19


Obtemos 26183 primeiras contribuições desde 2019. A partir disso vamos obter os demais dados importantes para contribuições

In [127]:
def get_total_commits(row):
    repo_id = row['project_id']
    date = row['created_at']
    total_commits = 0
    for chunk in df_commits:
        total_commits += chunk.loc[(chunk['project_id'] == repo_id) & (pd.to_datetime(chunk['created_at']) <= pd.to_datetime(date))].shape[0]

    return total_commits

In [128]:
df_first_commit_user['total_commits'] = df_first_commit_user.apply(get_total_commits, axis=1)   

C:\Users\gusge\AppData\Local\Temp\ipykernel_14952\2902815896.py:5: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in df_commits:
C:\Users\gusge\AppData\Local\Temp\ipykernel_14952\2902815896.py:5: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in df_commits:
C:\Users\gusge\AppData\Local\Temp\ipykernel_14952\2902815896.py:5: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in df_commits:
C:\Users\gusge\AppData\Local\Temp\ipykernel_14952\2902815896.py:5: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in df_commits:
C:\Users\gusge\AppData\Local\Temp\ipykernel_14952\2902815896.py:5: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in df_commits:
C:\Users\gusge\AppData\Local\T

Pegar os detalhes do projeto

In [23]:
df_projetos = pd.read_csv(join(path, "projects.csv"), names=["id", "url", "owner_id", "name", "descriptor", "language", "created_at", "forked_from", "deleted", "updated_at"], na_values="not available", on_bad_lines='warn', chunksize=5000000)